Usage:
- In Cell 1: 
    set stats_location to the location of the file "contig_statistics.txt" generated by the script SE_assembly.py
    set output_location to the folder where the output should be saved
- Run all cells of the notebook
- When a screen with the plot appears, you can place 6 dots to outline the contigs you want to select for the second part of the assembly. The selected contigs will be colored red on the plot when the last dot is placed
- The script will write the outputfile "hull_IDs"  containing the IDs of the selected contigs to selected folder


In [ ]:
stats_location = ""
output_location = ""

In [ ]:
import matplotlib as mpl
mpl.use('TkAgg')
import sys, os
import pandas as pd
import numpy as np
import scipy.spatial as sc
import matplotlib.pyplot as plt
from matplotlib.path import Path


In [ ]:
contig_data = pd.read_table(stats_location)
contig_data['average_coverage'] = np.log10(contig_data['average_coverage'])
selection = contig_data['length'] > 500
contig_data_red = contig_data[selection]
fig, ax = plt.subplots(1, figsize=(15,15))
ax.scatter(contig_data_red['GC'], contig_data_red['average_coverage'])
fig.show()

In [ ]:
hullpoints = plt.ginput(6, show_clicks=True)

In [ ]:
hullpoints = [list(x) for x in hullpoints]
print(hullpoints)

In [ ]:
hull = sc.ConvexHull(hullpoints)
for simplex in hull.simplices:
    print(simplex)
    print((hullpoints[simplex[0]][1], hullpoints[simplex[0]][0]))
    print((hullpoints[simplex[1]][1], hullpoints[simplex[1]][0]))
    ax.plot((hullpoints[simplex[0]][0], hullpoints[simplex[1]][0]), (hullpoints[simplex[0]][1], hullpoints[simplex[1]][1]), 'dk--')
    fig.show()

In [ ]:
hull_path = Path(hullpoints)
i = 0
for x,y in zip(contig_data_red['GC'], contig_data_red['average_coverage']):
    if hull_path.contains_point((x,y)):
        i += 1
        ax.scatter(x,y, color='red')
        fig.show()
print(i)

In [ ]:
ID_file = open(output_location+'/hull_IDs.txt', 'w')
for index, row in contig_data_red.iterrows():
    if hull_path.contains_point((row['GC'], row['average_coverage'])):
        print(row['contig'], file=ID_file)
ID_file.close()
        